In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as f 
from torch.autograd import Variable
torch.backends.cudnn.deterministic = True
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random 
from matplotlib import pyplot as plt
from sklearn import decomposition
from sklearn import manifold
from scipy import stats
from itertools import cycle

In [2]:
import sys
import utilities as ut
from Network import Net
import Network as nn

In [3]:
train_dataset, test_dataset = ut.load_files(dataset=1)
n = 5000#train_dataset.shape[0] 
epsilon = 0.1
input_size = train_dataset.shape[1]-1
hidden_size = 500
hidden_size2 = 500
num_classes = 2
num_epochs = 200
batch_size = 512
learning_rate = 0.001
learning_rate2 = 0.001
regularization = False
add_DR_based_data = True
train_dataset = train_dataset.sample(n)
samples = 5000

Freq1_harmonics_rel_phase_0
Freq2_harmonics_rel_phase_0
Freq3_harmonics_rel_phase_0


In [4]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [6]:
train_dataset, test_dataset = ut.delete_outliers(train_dataset, test_dataset)

(3878, 61)
(28625, 60)
(28625, 61)


In [7]:
train_dataset = ut.sort_columns(train_dataset)
test_dataset = ut.sort_columns(test_dataset)

In [8]:
test_dataset_pred = test_dataset.copy()
train_dataset_pred = train_dataset.copy()

In [9]:
data_prior = ut.generate_samples(samples, train_dataset, epsilon,  option = 2)

In [10]:
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.1, random_state=42)

In [11]:
train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.1, random_state=42)

In [12]:
_, _, train_target_prior, train_loader_prior = ut.get_tensors(train_dataset_prior, batch_size)
_, _, val_target_prior, val_loader_prior     = ut.get_tensors(val_dataset_prior, batch_size)
_, _, train_target, train_loader             = ut.get_tensors(train_dataset, batch_size)
_, _, train_target_pred, train_loader_pred   = ut.get_tensors(train_dataset_pred, batch_size)
_, _, val_target, val_loader                 = ut.get_tensors(val_dataset_prior, batch_size)
_, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)
_, _, test_target_pred, test_loader_pred     = ut.get_tensors(test_dataset_pred, batch_size)

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [13]:
net = Net(input_size, hidden_size, hidden_size2, num_classes)
net.cuda()

Net(
  (fc1): Linear(in_features=60, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=500, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=500, out_features=2, bias=True)
)

In [14]:
aux_loss_activated = True
num_epochs_prior = 2000
EPS1 = 1e-3
EPS2 = 1e-6
hist_val, hist_train = nn.train(net, train_loader, train_loader_prior, val_loader, EPS1, EPS2, learning_rate, input_size)

training: epoch:  1  loss:  0.6070810045514788
validating: epoch:  1  loss:  0.3905157744884491
training: epoch:  2  loss:  0.3659355810710362
validating: epoch:  2  loss:  0.15910480171442032
training: epoch:  3  loss:  0.2636608438832419
validating: epoch:  3  loss:  0.05512472242116928
training: epoch:  4  loss:  0.25302766263484955
validating: epoch:  4  loss:  0.022108664736151695
training: epoch:  5  loss:  0.2551100232771465
validating: epoch:  5  loss:  0.012518571224063635
training: epoch:  6  loss:  0.2454485446214676
validating: epoch:  6  loss:  0.009303504135459661
training: epoch:  7  loss:  0.23156024728502547
validating: epoch:  7  loss:  0.0077223568223416805
training: epoch:  8  loss:  0.22032369460378373
validating: epoch:  8  loss:  0.006339560030028224
training: epoch:  9  loss:  0.21265461828027452
validating: epoch:  9  loss:  0.004812728380784392
training: epoch:  10  loss:  0.2068947000162942
validating: epoch:  10  loss:  0.0033057316904887557
training: epoch:

training: epoch:  85  loss:  0.06480960441487175
validating: epoch:  85  loss:  7.578687927889405e-06
training: epoch:  86  loss:  0.06416353051151548
validating: epoch:  86  loss:  7.584782451885985e-06
training: epoch:  87  loss:  0.06354836427739688
validating: epoch:  87  loss:  7.514817298215348e-06
training: epoch:  88  loss:  0.06292863030518804
validating: epoch:  88  loss:  7.52334926801268e-06
training: epoch:  89  loss:  0.062338397439037053
validating: epoch:  89  loss:  7.4892200245813e-06
training: epoch:  90  loss:  0.061736512929201126
validating: epoch:  90  loss:  7.4892200245813e-06
training: epoch:  91  loss:  0.061171281550611765
validating: epoch:  91  loss:  7.432175152644049e-06
training: epoch:  92  loss:  0.0605992174574307
validating: epoch:  92  loss:  7.415841537294909e-06
training: epoch:  93  loss:  0.06005013308354786
validating: epoch:  93  loss:  7.3963392424047925e-06
training: epoch:  94  loss:  0.05950246378779411
validating: epoch:  94  loss:  7.36

training: epoch:  167  loss:  0.036401844184313505
validating: epoch:  167  loss:  5.642329142574454e-06
training: epoch:  168  loss:  0.036217955340232165
validating: epoch:  168  loss:  5.624289315164788e-06
training: epoch:  169  loss:  0.03603885881602764
validating: epoch:  169  loss:  5.612100267171627e-06
training: epoch:  170  loss:  0.03586196926023279
validating: epoch:  170  loss:  5.5879659157653805e-06
training: epoch:  171  loss:  0.035687796239342005
validating: epoch:  171  loss:  5.549935849558096e-06
training: epoch:  172  loss:  0.035513141325541904
validating: epoch:  172  loss:  5.547985892917495e-06
training: epoch:  173  loss:  0.03534118751330035
validating: epoch:  173  loss:  5.5270206757995766e-06
training: epoch:  174  loss:  0.03517291588442666
validating: epoch:  174  loss:  5.5087366490624845e-06
training: epoch:  175  loss:  0.035005038870232444
validating: epoch:  175  loss:  5.5050800256140064e-06
training: epoch:  176  loss:  0.034839323056595664
vali

validating: epoch:  248  loss:  4.841993813897716e-06
training: epoch:  249  loss:  0.02628553126539503
validating: epoch:  249  loss:  4.834680112253409e-06
training: epoch:  250  loss:  0.02620269837124007
validating: epoch:  250  loss:  4.831023488804931e-06
training: epoch:  251  loss:  0.026116632191198214
validating: epoch:  251  loss:  4.819078185391845e-06
training: epoch:  252  loss:  0.026040537282824516
validating: epoch:  252  loss:  4.81834695165162e-06
training: epoch:  253  loss:  0.02595292218029499
validating: epoch:  253  loss:  4.7910434659570456e-06
training: epoch:  254  loss:  0.025876719504594803
validating: epoch:  254  loss:  4.7944563448254485e-06
training: epoch:  255  loss:  0.025795855692454746
validating: epoch:  255  loss:  4.778854417963885e-06
training: epoch:  256  loss:  0.025719823581831797
validating: epoch:  256  loss:  4.770809482579352e-06
training: epoch:  257  loss:  0.025633820997817174
validating: epoch:  257  loss:  4.761545824294444e-06
tra

training: epoch:  330  loss:  0.021010848560503552
validating: epoch:  330  loss:  4.489241291594226e-06
training: epoch:  331  loss:  0.0209566666079419
validating: epoch:  331  loss:  4.483634256757796e-06
training: epoch:  332  loss:  0.02090413123369217
validating: epoch:  332  loss:  4.486315901885973e-06
training: epoch:  333  loss:  0.020849267153867652
validating: epoch:  333  loss:  4.475589776120614e-06
training: epoch:  334  loss:  0.020798771509102414
validating: epoch:  334  loss:  4.463888217287604e-06
training: epoch:  335  loss:  0.02074714989534446
validating: epoch:  335  loss:  4.4616940613195766e-06
training: epoch:  336  loss:  0.020689068096024648
validating: epoch:  336  loss:  4.442923000169685e-06
training: epoch:  337  loss:  0.020644267488803183
validating: epoch:  337  loss:  4.454137069842545e-06
training: epoch:  338  loss:  0.020586012596530572
validating: epoch:  338  loss:  4.425370661920169e-06
training: epoch:  339  loss:  0.020538680388459137
validat

...

training: epoch:  734  loss:  0.00840226308043514
validating: epoch:  734  loss:  1.6728305354263284e-06
training: epoch:  735  loss:  0.008380211291036435
validating: epoch:  735  loss:  1.6545469634365872e-06
training: epoch:  736  loss:  0.00836429964484913
validating: epoch:  736  loss:  1.6511339708813466e-06
training: epoch:  737  loss:  0.008343925100884267
validating: epoch:  737  loss:  1.6408951069024624e-06
training: epoch:  738  loss:  0.008329555857926607
validating: epoch:  738  loss:  1.6404076177423121e-06
training: epoch:  739  loss:  0.008311381829636437
validating: epoch:  739  loss:  1.6274871086352505e-06
training: epoch:  740  loss:  0.008290294956948076
validating: epoch:  740  loss:  1.6184671949304175e-06
training: epoch:  741  loss:  0.008276385920388358
validating: epoch:  741  loss:  1.6079845863714581e-06
training: epoch:  742  loss:  0.008255331244851862
validating: epoch:  742  loss:  1.603352757229004e-06
training: epoch:  743  loss:  0.00824055780789681

training: epoch:  815  loss:  0.007074276717113597
validating: epoch:  815  loss:  1.1284658967269934e-06
training: epoch:  816  loss:  0.007060155405529908
validating: epoch:  816  loss:  1.1226151173104881e-06
training: epoch:  817  loss:  0.007045368198305368
validating: epoch:  817  loss:  1.1165205933139077e-06
training: epoch:  818  loss:  0.007030104486537831
validating: epoch:  818  loss:  1.104575289900822e-06
training: epoch:  819  loss:  0.0070156691196773735
validating: epoch:  819  loss:  1.104087687053834e-06
training: epoch:  820  loss:  0.007000847586563655
validating: epoch:  820  loss:  1.0975055602102657e-06
training: epoch:  821  loss:  0.006987426429986954
validating: epoch:  821  loss:  1.0906796887866221e-06
training: epoch:  822  loss:  0.006972126197069883
validating: epoch:  822  loss:  1.085316512217105e-06
training: epoch:  823  loss:  0.006958499683865479
validating: epoch:  823  loss:  1.0821472642419394e-06
training: epoch:  824  loss:  0.0069422666648668

training: epoch:  895  loss:  0.0060120454457189354
validating: epoch:  895  loss:  6.981910019021598e-07
training: epoch:  896  loss:  0.005998880935034582
validating: epoch:  896  loss:  6.95021867613832e-07
training: epoch:  897  loss:  0.005987020741615977
validating: epoch:  897  loss:  6.884397407702636e-07
training: epoch:  898  loss:  0.005975412470953805
validating: epoch:  898  loss:  6.901462370478839e-07
training: epoch:  899  loss:  0.005964403173753193
validating: epoch:  899  loss:  6.789322242184426e-07
training: epoch:  900  loss:  0.005948957055807114
validating: epoch:  900  loss:  6.8161386934662e-07
training: epoch:  901  loss:  0.005940338031255773
validating: epoch:  901  loss:  6.740565936524945e-07
training: epoch:  902  loss:  0.005927404189216239
validating: epoch:  902  loss:  6.72106352794799e-07
training: epoch:  903  loss:  0.005914719575749976
validating: epoch:  903  loss:  6.691809630865464e-07
training: epoch:  904  loss:  0.005903716831068907
validat

training: epoch:  976  loss:  0.00512965748618756
validating: epoch:  976  loss:  4.241802855631249e-07
training: epoch:  977  loss:  0.005116871890744993
validating: epoch:  977  loss:  4.217424702801509e-07
training: epoch:  978  loss:  0.005105355521664023
validating: epoch:  978  loss:  4.217424702801509e-07
training: epoch:  979  loss:  0.005098687019199133
validating: epoch:  979  loss:  4.1442902443122875e-07
training: epoch:  980  loss:  0.005087984393217734
validating: epoch:  980  loss:  4.1467279743301333e-07
training: epoch:  981  loss:  0.00507749041675457
validating: epoch:  981  loss:  4.11016088719407e-07
training: epoch:  982  loss:  0.005067646137571761
validating: epoch:  982  loss:  4.0833447201293893e-07
training: epoch:  983  loss:  0.0050570970800306115
validating: epoch:  983  loss:  4.0857827343643294e-07
training: epoch:  984  loss:  0.005046235098104391
validating: epoch:  984  loss:  4.0540911072639574e-07
training: epoch:  985  loss:  0.005038743672360267
v

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
#plt.plot(aux_loss_behaviour)
#plt.show()

In [ ]:
nn.get_results(net, train_loader, input_size)
nn.get_results(net, test_loader, input_size)

In [ ]:
torch.save(net.state_dict(), 'model.pkl')

In [ ]:
plt.plot(hist_val, label ='validation')
plt.plot(hist_train, label ='train')
plt.legend()
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.savefig('images/'+str(samples)+'_'+str(epsilon)+'_'+str(n)+"_"+str(hidden_size)+"_Loss_Training.png")
plt.show()

In [ ]:
#csv_file = open("size_MLP_noise.csv", "a")
#csv_file.write(str(np.asarray(acc_testing))+","+str(np.asarray(acc_training))+","+str(samples)+","+str(epsilon)+","+str(n)+","+str(hidden_size)+"\n")
#csv_file.close()

In [ ]:
outputs, intermediates, intermediates2, labels = ut.get_representations(net, train_loader, device)
outputs_test, intermediates_test, intermediates2_test, labels_test = ut.get_representations(net, test_loader, device)

In [ ]:
intermediate_pca_data, intermediate_pca_data_test = ut.get_pca(intermediates, data_test=intermediates_test)
fig, axs = plt.subplots(2, 3, figsize=(20,15))
ut.plot_representations(intermediate_pca_data, labels, axs[0, 0])
ut.plot_representations(intermediate_pca_data_test, labels_test, axs[1, 0])
intermediate2_pca_data, intermediate2_pca_data_test = ut.get_pca(intermediates2, data_test=intermediates2_test)
ut.plot_representations(intermediate2_pca_data, labels, axs[0, 1])
ut.plot_representations(intermediate2_pca_data_test, labels_test, axs[1, 1])
output_pca_data, output_pca_data_test = ut.get_pca(outputs, data_test=outputs_test)
ut.plot_representations(output_pca_data, labels, axs[0, 2])
ut.plot_representations(output_pca_data_test, labels_test, axs[1, 2])

In [ ]:
N_CURVES = 25000

fig, axs = plt.subplots(2, 3, figsize=(20,15))
intermediate_tsne_data, intermediate_tsne_data_test = ut.get_tsne(intermediates, data_test= intermediates_test, n_curves = N_CURVES)
ut.plot_representations(intermediate_tsne_data, labels, axs[0, 0],  n_curves = N_CURVES)
ut.plot_representations(intermediate_tsne_data_test, labels_test, axs[1, 0], n_curves = N_CURVES)

intermediate2_tsne_data, intermediate2_tsne_data_test = ut.get_tsne(intermediates2, data_test=intermediates2_test, n_curves = N_CURVES)
ut.plot_representations(intermediate2_tsne_data, labels, axs[0, 1], n_curves = N_CURVES)
ut.plot_representations(intermediate2_tsne_data_test, labels_test, axs[1, 1], n_curves = N_CURVES)

output_tsne_data, output2_tsne_data_test = ut.get_tsne(outputs, data_test=outputs_test, n_curves = N_CURVES)
ut.plot_representations(output_tsne_data, labels, axs[0, 2], n_curves = N_CURVES)
ut.plot_representations(output2_tsne_data_test, labels_test, axs[1, 2], n_curves = N_CURVES)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,15))
curves, labels, probs_train = ut.get_predictions(net, train_loader_pred, device)
pred_labels = probs_train.argmax(1, keepdim = True)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax1)
curves, labels, probs_test = ut.get_predictions(net, test_loader_pred, device)
pred_labels = probs_test.argmax(1, keepdim = True)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax2)

In [ ]:
curves, labels, probs_train_sample = ut.get_predictions(net, train_loader, device)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,15))
ax1.hist(probs_train[:,0], color='black')
ax1.set_xlabel('training set')
ax2.hist(probs_train_sample[:,0], color='black')
ax2.set_xlabel('training set + samples')
ax3.hist(probs_test[:,0], color='black')
ax3.set_xlabel('testing set')
plt.show()

In [ ]:
#N_WEIGHTS = 25
#weights = net.fc2.weight.data
#plot_weights(weights, N_WEIGHTS)

In [ ]:
weights1 = net.fc1.weight.data
w1 = weights1.cpu().numpy().reshape(-1,1)
weights2 = net.fc2.weight.data
w2 = weights2.cpu().numpy().reshape(-1,1)
weights3 = net.fc3.weight.data
w3 = weights3.cpu().numpy().reshape(-1,1)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,15))
ax1.hist(w1, color='black')
ax1.set_xlabel('Layer 1')
ax2.hist(w2, color='black')
ax2.set_xlabel('Layer 2')
ax3.hist(w3, color='black')
ax3.set_xlabel('Layer 3')
plt.show()